# MNE (MEG + EEG analysis & visualization)

[Main Homepage](https://mne-tools.github.io/stable/index.html)

**Note:** Do not forget to install scipy version 0.19 (until scipy version 2.0.1 is released).

## Supporte data formats

Datatype            | File format              | Extension | MNE-Python function
--------------------|--------------------------|-----------|------------------------
MEG                 | Elekta Neuromag          | .fif      | `mne.io.read_raw_fif()`
MEG                 | 4-D Neuroimaging / BTI   | dir       | `mne.io.read_raw_bti()`
MEG                 | CTF                      | dir       | `mne.io.read_raw_ctf()`
MEG                 | KIT                      | sqd       | `mne.io.read_raw_kit()` and `mne.read_epochs_kit()`
EEG                 | Brainvision              | .vhdr     | `mne.io.read_raw_brainvision()`
EEG                 | Neuroscan CNT            | .cnt      | `mne.io.read_raw_cnt()`
EEG                 | European data format     | .edf      | `mne.io.read_raw_edf()`
EEG                 | Biosemi data format      | .bdf      | `mne.io.read_raw_edf()`
EEG                 | General data format      | .gdf      | `mne.io.read_raw_edf()`
EEG                 | EGI simple binary        | .egi      | `mne.io.read_raw_egi()`
EEG                 | EGI MFF format           | .mff      | `mne.io.read_raw_egi()`
EEG                 | EEGLAB                   | .set      | `mne.io.read_raw_eeglab()` and `mne.read_epochs_eeglab()`
Electrode locations | elc, txt, csd, sfp, htps | Misc      | `mne.channels.read_montage()`
Electrode locations | EEGLAB loc, locs, eloc   | Misc      | `mne.channels.read_montage()`

For more see [here](https://mne-tools.github.io/stable/manual/io.html#importing-meg-data).

# Import relevant packages

In [ ]:
%pylab inline
import mne

# Specify file to use

This is just an example file, downsampled to 100Hz, so don't get all crazy and critical on it.

In [ ]:
fname = 'oddball_example-raw.fif'

# Read in raw data / raw objects

In [ ]:
raw = mne.io.read_raw_fif(fname, preload=True)

Let's look at the data:

In [ ]:
raw

In [ ]:
%matplotlib notebook
raw.plot();

In [ ]:
' '.join(raw.ch_names)

In [ ]:
raw.info

In [ ]:
raw.info['bads']

# Reference to average

Now, let's set the reference to the average. (*Bad EEG channels are automatically excluded if they are properly set in ``info['bads']``.*)

In [ ]:
raw.set_eeg_reference('average', projection=False)

# Filter the data

Let's say we want to filter the data between 1Hz and 20 Hz.

In [ ]:
raw.filter(1, 20)

In [ ]:
%matplotlib notebook
raw.plot();

In [ ]:
raw.info['bads']

# Remove EOG artefacts

There are many EOG artefacts. We will use ICA to correct these. For this, we create an ICA object
and use its `.fit` method on a filtered copy of the raw data:

In [ ]:
n_components = 20
ica = mne.preprocessing.ICA(n_components=n_components, random_state=0)

In [ ]:
ica.fit(raw.copy().filter(8, 35))

In [ ]:
%matplotlib inline
ica.plot_components(outlines="skirt");

In [ ]:
ica.plot_properties(raw, picks=[8]);

## Automatic Artifact Correction

For more investigations and cool automatic ICA classification, see: https://www.martinos.org/mne/dev/auto_tutorials/plot_artifacts_correction_ica.html#advanced-artifact-detection

In [ ]:
from mne.preprocessing import create_eog_epochs

picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                           stim=False, exclude='bads')

Get single EOG trials:

In [ ]:
eog_average = create_eog_epochs(raw, ch_name='FP1', picks=picks_eeg)

In [ ]:
eog_average = eog_average.average()
eog_average

In [ ]:
%matplotlib inline
ica.plot_sources(eog_average, exclude=[6, 13]);

In [ ]:
eog_average.plot_joint(times=np.linspace(-0.5, 0.5, 5));

In [ ]:
eog_average.plot_image();

In [ ]:
eog_average.plot_topo();

In [ ]:
eog_average.plot_topomap(times=np.linspace(-0.5, 0.5, 5));

In [ ]:
artifacts = ica.detect_artifacts(raw.copy(), eog_ch='FP1', eog_criterion=0.5)

In [ ]:
artifacts.exclude

## Exclude bad components

We store "bad" components in the ica object.

In [ ]:
ica.exclude = artifacts.exclude

Let's compare raw and corrected data ...

In [ ]:
%matplotlib inline

In [ ]:
raw.plot();

In [ ]:
ica_out = ica.apply(raw.copy(), exclude=ica.exclude)

In [ ]:
ica.apply(ica_out).plot();

# Epochs

For epoching the data, we need event markers. Usually, these are stored in the `raw` object.

In [ ]:
events = mne.find_events(ica_out)

`events` is simply an array (time in samples, zero, trigger);

In [ ]:
events

In [ ]:
mne.viz.plot_events(events[:100]);

For creating an `mne.Epochs` object, we require, in addition to the `raw` object and the `events` array, a dictionary of the intended condition names and the corresponding trigger numbers.

In [ ]:
event_ids = {"standard/stimulus": 200, "target/stimulus": 100}

In [ ]:
epochs_pre = mne.Epochs(ica_out.copy(), events, event_id=event_ids, preload=True, reject=dict(eeg=80e-6))

In [ ]:
print('%.2f%% of epochs were dropped' % epochs_pre.drop_log_stats())

In [ ]:
bad_channels = np.sort([x for x in flatten([d for d in epochs_pre.drop_log if d != []])])
bad_channels

In [ ]:
channel_appearance = [(c, np.sum(bad_channels==c)) for c in np.unique(bad_channels)]
channel_appearance

In [ ]:
kick_out_threshold = 8
exclude_channels = [c[0] for c in channel_appearance if c[1]>=kick_out_threshold]
exclude_channels

In [ ]:
ica_out.info['bads'] = exclude_channels

In [ ]:
ica_out.interpolate_bads(reset_bads=True)

In [ ]:
epochs = mne.Epochs(ica_out, events, event_id=event_ids, preload=True, reject=dict(eeg=80e-6))

In [ ]:
print('%.2f%% of epochs were dropped' % epochs.drop_log_stats())

In [ ]:
epochs.plot();

Plot average curve:

In [ ]:
epochs.average().plot(titles=dict(eeg='Average reference'), show=False);

The `mne.Epochs` constructor has a number of options, such as time window lengths and rejection thresholds.
Investigate them on your own.

# Baseline Correction
`Epochs` objects also have various methods, different from `raw` objects - e.g., for baselining.

In [ ]:
epochs = epochs.apply_baseline((None, 0))
epochs

# Access only specific conditions

To subselect only a sample of epochs, a dict-like access mode is available.

In [ ]:
epochs["target"]

Observe how tags selected by forward slashes - "/" - work.

In [ ]:
epochs["stimulus"]

# Plot specific electrode

In [ ]:
epochs["target"].plot_image(picks=[13]);

# Plot GFP

In [ ]:
epochs["target"].plot_image(combine='gfp', cmap="YlGnBu_r");

# Event related potentials (ERP)

Finally, if we average an epoched dataset over trials, we can use the `mne.Evoked` object.

In [ ]:
target = epochs["target"].average()
target

In [ ]:
standard = epochs["standard"].average()

To quickly investigate evoked activity, the `Evoked` object has a number of plotting functions available.

In [ ]:
target.plot_joint(times='peaks', ts_args=dict(gfp=True));

In [ ]:
standard.plot_joint(times='peaks', ts_args=dict(gfp=True));

# Differences between two conditions

For condition contrasts, you can use `mne.combine.evoked`:

In [ ]:
difference = mne.combine_evoked((target, standard), weights=(.5, -.5))

In [ ]:
difference.plot_joint(times=.35, ts_args=dict(gfp=True));

In [ ]:
difference.plot_topomap(times='peaks');

To contrast multiple conditions, `mne.viz.plot_compare_evokeds` is available:

In [ ]:
mne.viz.plot_compare_evokeds({"target": target, "standard": standard},
                             truncate_yaxis=True);

# Permutation F-test on sensor data with 1D cluster level
According to: https://martinos.org/mne/stable/auto_examples/stats/plot_cluster_stats_evoked.html#sphx-glr-auto-examples-stats-plot-cluster-stats-evoked-py

In [ ]:
from mne.stats import permutation_cluster_test

fdr_epoch = epochs.copy()
fdr_epoch.drop_channels(['STI 014'])

epochs1 = fdr_epoch['target']
epochs2 = fdr_epoch['stimulus']

condition1 = epochs1.get_data().std(1)  # GFP as 3D matrix
condition2 = epochs2.get_data().std(1)  # GFP as 3D matrix

In [ ]:
threshold = 4
T_obs, clusters, cluster_p_values, H0 = permutation_cluster_test(
    [condition1, condition2], n_permutations=10000, threshold=threshold, tail=1, n_jobs=1)

In [ ]:
times = epochs['target'].times
plt.close('all')
plt.figure(figsize=(12, 6))
plt.subplot(211)
plt.title('GFP')
plt.plot(times, condition1.mean(axis=0) - condition2.mean(axis=0),
         label="GFP Contrast (Cond 1 - Cond 2)")
plt.ylabel("GFP")
plt.legend()
plt.subplot(212)
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] <= 0.05:
        h = plt.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
    else:
        plt.axvspan(times[c.start], times[c.stop - 1], color=(0.3, 0.3, 0.3),
                    alpha=0.3)
hf = plt.plot(times, T_obs, 'g')
plt.legend(['cluster p-value < 0.05'])
plt.xlabel("time (ms)")
plt.ylabel("f-values")
plt.show()

## MVPA/decoding

Can we predict trial type from EEG activity?

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

To speed up and simplify things, we balance the epochs counts for the two trials.

In [ ]:
epochs.equalize_event_counts(event_ids);

In [ ]:
epochs

In [ ]:
epochs.pick_types(eeg=True)
X = epochs.get_data()  # features
y = epochs.events[:, -1] == event_ids["target/stimulus"]  # targets
X.shape, y.shape

X has the wrong shape - `samples`, `channels`, `times`, should be `samples`, `features`.

We can use `mne.decoding.Vectorizer` to correctly shape the data. It fits right into a scikit-learn pipeline.

In [ ]:
from mne.decoding import Vectorizer

In [ ]:
clf = make_pipeline(Vectorizer(), StandardScaler(), LinearSVC(class_weight="balanced"))
res = cross_val_score(clf, X, y, cv=8)  # accuracy
print(np.round(res.mean() * 100, 3))
print(res)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = make_pipeline(Vectorizer(), StandardScaler(), LogisticRegression(class_weight="balanced"))
res = cross_val_score(clf, X, y, cv=8)  # accuracy
print(np.round(res.mean() * 100, 3))
print(res)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = make_pipeline(Vectorizer(), StandardScaler(), RandomForestClassifier(class_weight="balanced"))
res = cross_val_score(clf, X, y, cv=8)  # accuracy
print(np.round(res.mean() * 100, 3))
print(res)

# Kind of Searchlight - Classification over time

At which time points in the trial is there information about trial category?

We need two more tools for this: one to train and score at each time point, and one to handle the cross-validated scoring for the former.

In [ ]:
from mne.decoding import SlidingEstimator, cross_val_multiscore, LinearModel, get_coef
clf = make_pipeline(Vectorizer(), StandardScaler(), LinearModel(LinearSVC(class_weight="balanced")))
sl = SlidingEstimator(clf, scoring='roc_auc')

In [ ]:
scores_time_decoding = cross_val_multiscore(sl, X, y, cv=8)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(epochs.times, scores_time_decoding.mean(0), label='score')
ax.axhline(.5, color='k', linestyle='-', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='--')
ax.set_title('Sensor space decoding')
plt.show()

In [ ]:
# You can retrieve the spatial filters and spatial patterns if you explicitly use a LinearModel
sl.fit(X, y)
coef = get_coef(sl, 'patterns_', inverse_transform=True)
evoked = mne.EvokedArray(coef, epochs.info, tmin=epochs.times[0])
evoked.plot_joint(times='peaks', title='patterns');

But is the same thing happening at each time point? We can investigate that with generalization across time decoding.

In [ ]:
from mne.decoding import GeneralizingEstimator
gen = GeneralizingEstimator(clf)
scores_gat = cross_val_multiscore(gen, X, y)

In [ ]:
scores_gat.shape

In [ ]:
import numpy as np
data = scores_gat.mean(0)
vmax = np.abs(data).max()
tmin, tmax = epochs.times[[0, -1]]

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(
    data,
    origin="lower", cmap="RdBu_r",
    extent=(tmin, tmax, tmin, tmax),
    vmax=vmax, vmin=1-vmax);

ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('Temporal Generalization')
ax.axvline(0, color='k')
ax.axhline(0, color='k')

plt.colorbar(im)

Horizontal line means. If we take a classifier at timepoint (x,y), how well does this predict the timepoint x ms later?

So in this figure it means. Around the middle blob (around 180ms), something else happens that in the upper right corner (450ms).

## Time-Frequency stuff

For an overview over the spectral shape of the data, we can use a plotting method of `raw`, `raw.plot_psd`:

In [ ]:
raw.plot_psd(fmin=1, fmax=20);

The very green high curve is actually the eye blinks.

In [ ]:
ica.apply(raw.copy(), exclude=[0, 6]).plot_psd(fmin=1, fmax=20);

This figure above is not specific to time. It overall time points.

But what about the time/frequency correlates of the Oddball effect?

We will extract power per time and frequency with Morlet wavelets.

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
epochs_for_tfr = mne.Epochs(raw, events, event_id=event_ids,
                            tmin=-.5, tmax=1.5, preload=True)  # need longer data segment
epochs_for_tfr = ica.apply(epochs_for_tfr, exclude=ica.exclude)
epochs_for_tfr.equalize_event_counts(event_ids);  # to speed up things

In [ ]:
freqs = np.arange(3, 30)
tfr_target = tfr_morlet(epochs_for_tfr["target"], freqs, 3, return_itc=False)
tfr_standard = tfr_morlet(epochs_for_tfr["standard"], freqs, 3, return_itc=False)

In [ ]:
tfr_target

In [ ]:
tfr_standard

Time-frequency data (single trial or averaged) is stored in TFR objects. These objects behave in many ways like Evoked objects ...

In [ ]:
tfr = mne.combine_evoked((tfr_standard, tfr_target), (-.5, .5))
tfr.apply_baseline((None, 0))

Plotting time-frequencyy activity (event-related spectral perturbations): observe the alpha-band ERD and the time-frequency correlates of the P3 effect.

In [ ]:
tfr.plot(picks=[27]);

So what we see is a contrast between the standards and targets. So it means, that in the targets, the alpha activity goes down at 400ms in contrast to standard stimuli.

## Statistics

### Cluster-based permutation stats

Exploratory analysis with nonparametric control of the error rate is commonly done with
cluster-based permutation tests (i.e., Maris 2012). To cluster across space, we first need a
channel adjacency matrix.

In [ ]:
from mne.channels import find_ch_connectivity
connectivity, ch_names = find_ch_connectivity(epochs.info, ch_type='eeg')
plt.imshow(connectivity.toarray(), cmap="Greys")

To see which channels are adjacent.

Now we need the data in the right shape. Sadly, because the space dimension needs
to be last, we need to manually swap the time and space axes.

In [ ]:
epochs.pick_types(eeg=True)
target_epochs, standard_epochs = epochs["target"].get_data(), epochs["standard"].get_data()
target_epochs.shape, standard_epochs.shape

In [ ]:
target_epochs = target_epochs.swapaxes(1, 2)
standard_epochs = standard_epochs.swapaxes(1, 2)
target_epochs.shape, standard_epochs.shape

MNE has various cluster-based permutation test options. Here, we test for single-trial
differences between conditions with `mne.stats.spatio_temporal_cluster_test`.

We use threshold-free cluster enhancement to reduce the number of parameters.

Warning: the next cell takes a lot of time and computational power.

In [ ]:
from mne.stats import spatio_temporal_cluster_test

tfce = dict(start=.1, step=.3)
cluster_stats = spatio_temporal_cluster_test([target_epochs, standard_epochs],
                                             threshold=tfce,
                                             n_permutations=256,
                                             n_jobs=-1,
                                             connectivity=connectivity)
T_obs, clusters, p_values, _ = cluster_stats

Now we can visualise the *t* values over time and space ...

In [ ]:
extent = (*epochs.times[[0, -1]], 0, len(epochs.ch_names))
im = plt.imshow(T_obs.T, aspect="auto", cmap="RdBu_r",
                vmin=-100, vmax=100, extent=extent
          )
plt.colorbar(im)

... and the p-values.

In [ ]:
plt.hist(p_values)

alpha = .01
print(sum(p_values < alpha))

In [ ]:
plt.imshow(p_values.reshape(T_obs.shape).T < alpha, aspect="auto", cmap="Reds",
           extent=extent)

In [ ]:
plot((p_values.reshape(T_obs.shape).T < alpha).mean(0) * 100)

### Parametric stats
Sometimes, e.g. because we wish to test a specific hypothesis, cluster-based permutation tests are too much.
We can also simply access the data in array form and test with parametric (or nonparametric) tests.

For this, we first need to identify the spatial and temporal coordinates of an effect we want to test -
for example, the N2 at Cz.

In [ ]:
time_mask = (.2 < epochs.times) & (epochs.times < .25)
plt.plot(time_mask)

Now we extract the target data. Reminder: the shape of epochs data is (trial, channel, time)

In [ ]:
epochs["target"].get_data().shape

In [ ]:
cond_a = epochs["target"].get_data()[:, :, time_mask].std(1).mean(-1)
cond_b = epochs["standard"].get_data()[:, :, time_mask].std(1).mean(-1)

Now we can simply use ordinary tests on these statistics.

In [ ]:
from scipy.stats import ttest_ind, wilcoxon

In [ ]:
ttest_ind(cond_a, cond_b)

In [ ]:
wilcoxon(cond_a, cond_b)

It is also straight-forward to convert the data into a (pandas) dataframe.

In [ ]:
df = epochs.to_data_frame()
df.head(20)

In [ ]:
df_cz = df.query("200 < time < 250")["Cz"].groupby(["epoch", "condition"]).mean().reset_index()
df_cz.head()

In [ ]:
import seaborn as sns
sns.factorplot(y="Cz", data=df_cz, x="condition")

# Supplementary

- [Sort your ERP by reaction time (RT)](https://mne-tools.github.io/stable/auto_examples/visualization/plot_roi_erpimage_by_rt.html#sphx-glr-auto-examples-visualization-plot-roi-erpimage-by-rt-py)
- [FDR correction on T-test on sensor data](https://mne-tools.github.io/stable/auto_examples/stats/plot_fdr_stats_evoked.html)
- [Permutation t-test on source data with spatio-temporal clustering](https://mne-tools.github.io/stable/auto_tutorials/plot_stats_cluster_spatio_temporal.html)
- [Compute effect-matched-spatial filtering (EMS)](https://martinos.org/mne/stable/auto_examples/decoding/plot_ems_filtering.html#sphx-glr-auto-examples-decoding-plot-ems-filtering-py)
- [Plotting the full MNE solution](https://mne-tools.github.io/stable/auto_examples/inverse/plot_vector_mne_solution.html#sphx-glr-auto-examples-inverse-plot-vector-mne-solution-py)
- [XDAWN Denoising](https://mne-tools.github.io/stable/auto_examples/preprocessing/plot_xdawn_denoising.html)